# Data exploration

In [217]:
import pandas as pd

# Processing the existing columns

## Splitting the existing columns

In [218]:
PATH_FILE = "./data/raw_materials.csv"
data = pd.read_csv(PATH_FILE, delimiter=' \t', engine='python')

data[['Campaign', 'Level']] = data['Location'].str.rsplit(' ', n=1, expand=True)
data[['Campaign', 'Variation']] = data['Campaign'].str.rsplit(' ', n=1, expand=True)
data['Variation'].fillna('Standard', inplace=True)

data['Chance 2'].fillna(0, inplace=True)

## Fixing level column (NaN and transform to int column)

In [219]:
data.drop(data.loc[data.Level.isnull()].index, inplace=True) # Drop NaN (Azrael, not in campaign)

In [220]:
data['Level'] = data['Level'].astype(int)

## Fixing Fall of Cadia name

In [221]:
data.loc[data['Campaign'] == 'Fall of', ['Campaign', 'Variation']] = ['Fall of Cadia', 'Standard']

## Check

In [222]:
data

,Upgrade,Rarity,Stat,Location,Chance 1,Chance 2,Campaign,Level,Variation
0,Adamantium Filaments,Common,Health,Indomitus 59,75.0,0.0,Indomitus,59,Standard
1,Adamantium Filaments,Common,Health,Octarius 7,75.0,0.0,Octarius,7,Standard
2,Adamantium Filaments,Common,Health,Indomitus Mirror 47,83.3,0.0,Indomitus,47,Mirror
3,Adamantium Filaments,Common,Health,Octarius Elite 11,100.0,40.0,Octarius,11,Elite
4,Adamantium Ore,Common,Armour,Indomitus 48,75.0,0.0,Indomitus,48,Standard
...,...,...,...,...,...,...,...,...,...
502,Writings,Uncommon,Health,Indomitus Elite 12,100.0,17.0,Indomitus,12,Elite
503,Zeidos Ribbon,Rare,Health,Fall of Cadia 67,33.3,0.0,Fall of Cadia,67,Standard
504,Zeidos Ribbon,Rare,Health,Indomitus Mirror 16,40.0,0.0,Indomitus,16,Mirror
505,Zeidos Ribbon,Rare,Health,Fall of Cadia Mirror 43,40.0,0.0,Fall of Cadia,43,Mirror


# Adding more comfortable columns

## Fix the drop rate in a single column

In [223]:
data['Chance 1'] = data['Chance 1'] / 100
data['Chance 2'] = data['Chance 2'] / 100

data.rename(columns={"Chance 1": "Chance 1 (%)", "Chance 2": "Chance 2 (%)"}, inplace=True)

In [224]:
data.columns

Index(['Upgrade', 'Rarity', 'Stat', 'Location', 'Chance 1 (%)', 'Chance 2 (%)',
       'Campaign', 'Level', 'Variation'],
      dtype='object')

In [225]:
data['Drop rate (%)'] = data['Chance 1 (%)'] + data['Chance 2 (%)']

## Fix the cost

In [227]:
# Indomitus up to 29 is 5 energy
energy_cost = {
    'Tutorial': 5, 
    'Standard': 6,
    'Elite': 10,
}

In [226]:
data['Cost'] = 6
data.loc[data['Variation'] == 'Elite', 'Cost'] = 10

In [228]:
con = (data['Campaign'] == 'Indomitus') & (data['Variation'] == 'Standard')& (data['Level'] <= 29)
data.loc[con, 'Cost'] = 5

In [229]:
data.loc[data['Cost'] == 5].sort_values('Level')

,Upgrade,Rarity,Stat,Location,Chance 1 (%),Chance 2 (%),Campaign,Level,Variation,Drop rate (%),Cost
442,The Iron Phoenix,Common,Health,Indomitus 16,0.75,0.0,Indomitus,16,Standard,0.75,5
195,Frag Grenade,Common,Damage,Indomitus 17,0.75,0.0,Indomitus,17,Standard,0.75,5
341,Purity Seal,Common,Armour,Indomitus 19,0.75,0.0,Indomitus,19,Standard,0.75,5
167,Enriched Rations,Uncommon,Health,Indomitus 20,0.60,0.0,Indomitus,20,Standard,0.60,5
62,Basic Auxiliary Core,Common,Damage,Indomitus 22,0.75,0.0,Indomitus,22,Standard,0.75,5
437,The Gold Phoenix,Uncommon,Health,Indomitus 23,0.60,0.0,Indomitus,23,Standard,0.60,5
41,Attack Vector Data-Slate,Uncommon,Damage,Indomitus 25,0.60,0.0,Indomitus,25,Standard,0.60,5
123,Ceramite Lump,Uncommon,Armour,Indomitus 26,0.60,0.0,Indomitus,26,Standard,0.60,5
322,Pile of Salvage,Common,Armour,Indomitus 28,0.75,0.0,Indomitus,28,Standard,0.75,5


## Average energy cost

In [230]:
data['Return per energy'] = data['Drop rate (%)'] / data['Cost']  

# Save the file

In [231]:
data = data[['Upgrade', 'Rarity', 'Stat', 
            'Campaign', 'Variation', 'Level',
            'Drop rate (%)', 'Cost', 'Return per energy']]

In [232]:
data

,Upgrade,Rarity,Stat,Campaign,Variation,Level,Drop rate (%),Cost,Return per energy
0,Adamantium Filaments,Common,Health,Indomitus,Standard,59,0.750,6,0.125000
1,Adamantium Filaments,Common,Health,Octarius,Standard,7,0.750,6,0.125000
2,Adamantium Filaments,Common,Health,Indomitus,Mirror,47,0.833,6,0.138833
3,Adamantium Filaments,Common,Health,Octarius,Elite,11,1.400,10,0.140000
4,Adamantium Ore,Common,Armour,Indomitus,Standard,48,0.750,6,0.125000
...,...,...,...,...,...,...,...,...,...
502,Writings,Uncommon,Health,Indomitus,Elite,12,1.170,10,0.117000
503,Zeidos Ribbon,Rare,Health,Fall of Cadia,Standard,67,0.333,6,0.055500
504,Zeidos Ribbon,Rare,Health,Indomitus,Mirror,16,0.400,6,0.066667
505,Zeidos Ribbon,Rare,Health,Fall of Cadia,Mirror,43,0.400,6,0.066667


In [234]:
PATH_SAVE = './data/raw_materials_cleaned.csv'
data.to_csv(PATH_SAVE)